In [ ]:
import vk_api
import re
import pymorphy2 as pm
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
def inputlink():
    
    regex = r"vk\.com\/[0-9A-Za-z.]+"
    result = None
    while result == None:
        a = input('Введите ссылку на группу VK в формате vk.com/xxx, где xxx - id группы: ')
        result = re.match(regex, a)
        
    return a

In [ ]:
def subst(link):
    
    regex = r"(vk\.com\/)"
    subst = ""
    result = re.sub(regex, subst, link, 0, re.MULTILINE)
    if result:
        return result

In [ ]:
def import_posts(name):
    
    #вставить свой токен
    session = vk_api.VkApi(token='f6aaa32db12abd2d31ba1d64ac970d405f5e12fd2c9f01ee25b6bc83c017e3280ab7034cc1998918aec40')
    vk = session.get_api()
    offset = 0
    c = 0
    post = []
    for i in range(0,50):
        data = vk.wall.get(domain=name, count=100, offset=offset)
        offset += 50
        for i in data['items']:
            post.append(i['text'])
            c += 1
            #sleep(20)
    with open('posts.txt','w') as f:
        for each in post:
            try:
                f.write(each)
            except UnicodeEncodeError:
                pass
    with open('posts.txt', 'r') as file:
        posts = file.read()
        regex = r"[.?!,@#$%^&*()\"№:;\-+<>/\]\[1-9]]"
        posts = re.sub(regex, '', posts)
        regex = r"(https?:\/\/)?([\da-z\.-]+)\.([a-z\.]{2,6})([\/\w \.-]*)*\/?"
        posts = re.sub(regex, '', posts)
        wordsinposts = posts.split()
        
    return wordsinposts

In [ ]:
def choosePOS():
    
    table = {'NOUN': 'имя существительное', 'ADJ': 'имя прилагательное',
             'VERB': 'глагол', 'PRT': 'причастие', 'GRND': 'деепричастие'}
    for key, value in table.items():
        print(key, '-->', value)
    pos = input('Укажите часть речи, выбрав из левой колонки необходимое сокращение: ')
    while pos.upper() not in table.keys():
        pos = input('Такая часть речи не найдена, попробуйте ещё раз: ')
    print('Пожалуйста, выберите теги, по которым надо построить тепловую карту. После того, как выберете все нужные теги, нажмите Enter:  ')
    if pos.lower() == 'noun':
        print('''          1 — одушевлённость
          2 — род
          3 — число
          4 — падеж
          Введите цифры: ''')
        num = 4
    if pos.lower() == 'adj':
        print('''          1 — полнота/краткость 
          2 — род
          3 — число
          4 — падеж
          Введите цифры: ''') 
        num = 4
    if pos.lower() == 'verb':
        print('''          1 — аспект (вид)
          2 — переходность
          3 — число
          4 — лицо
          5 — время
          6 — наклонение
          Введите цифры: ''')
        num = 6
    if pos.lower() == 'grnd':
        print('''          1 — аспект (вид)
          2 — переходность
          3 — время
          Введите цифры: ''')
        num = 3
    if pos.lower() == 'prt':
        print('''          1 — полнота/краткость
          2 — аспект (вид)
          3 — время
          4 — действительное/страдательное
          5 — число
          6 — род
          7 — падеж (только для полных причастий)
          Введите цифры: ''')
        num = 7
    tags = []
    while True:
        a = input()
        if a == '':
            if len(tags)>1:
                break
        else:
            try:
                tags.append(int(a))
            except ValueError:
                print('пожалуйста, введите цифру: ')
            if tags != [] and tags[-1] > num:
                tags.pop()
                print('пожалуйста, введите цифру от 1 до',num)
            
        
    return pos, tags

In [ ]:
def chooseNOUN(p, tags):
    
    combination = []
    for tagn in tags:
        if tagn == 1:
            combination.append(p.tag.animacy)            
        if tagn == 2:
            combination.append(p.tag.gender)
        if tagn == 3:
            combination.append(p.tag.number)
        if tagn == 4:
            combination.append(p.tag.case)
            
    return combination

In [ ]:
def chooseADJ(p,tags):
    
    combination = []
    for tagn in tags:
        if tagn == 1:
            if p.tag.POS == 'ADJF':
                combination.append('full')
            else:
                combination.append('short')
        if tagn == 2:
            combination.append(p.tag.gender)
        if tagn == 3:
            combination.append(p.tag.number)
        if tagn == 4:
            combination.append(p.tag.case)
            
    return combination

In [ ]:
def chooseVERB(p, tags):
    
    combination = []
    for tagn in tags:
        if tagn == 1:
            combination.append(p.tag.aspect)
        if tagn == 2:
            combination.append(p.tag.transitivity)
        if tagn == 3:
            combination.append(p.tag.number)
        if tagn == 4:
            combination.append(p.tag.person)
        if tagn == 5:
            combination.append(p.tag.tense)
        if tagn == 6:
            combination.append(p.tag.mood)
            
    return combination

In [ ]:
def chooseGRND(p,tags):
    
    combination = []
    for tagn in tags:
        if tagn == 1:
            combination.append(p.tag.aspect)
        if tagn == 2:
            combination.append(p.tag.transitivity)
        if tagn == 3:
            combination.append(p.tag.tense)
            
    return combination

In [ ]:
def choosePRT(p,tags):
    
    combination = []
    for tagn in tags:
        if tagn == 1:
            if p.tag.POS == 'PRTF':
                combination.append('full')
            else:
                combination.append('short')
        if tagn == 2:
            combination.append(p.tag.aspect)
        if tagn == 3:
            combination.append(p.tag.tense)
        if tagn == 4:
            combination.append(p.tag.voice)
        if tagn == 5:
            combination.append(p.tag.number)
        if tagn == 6:
            combination.append(p.tag.gender)
        if tagn == 7:
            combination.append(p.tag.case)
            
    return combination

In [ ]:
def listoftags(pos, wordsinposts, tags, morph):
    
    listofcombos = []
    DFcolind = []
    for word in wordsinposts:
        scores = []
        p = morph.parse(word)
        for i in p:
            scores.append(i.score)
        n = scores.index(max(scores))
        p = morph.parse(word)[n]
        if pos.upper() == 'ADJ' and (p.tag.POS == 'ADJF' or p.tag.POS == 'ADJS'):
            a = chooseADJ(p, tags)
            listofcombos.append(a)
        elif pos.upper() == 'PRT' and (str(p.tag.POS) == 'PRTF' or str(p.tag.POS) == 'PRTS'):
            a = choosePRT(p, tags)
            listofcombos.append(a)
        elif str(p.tag.POS) == pos.upper():
            if pos.upper() == 'NOUN':
                a = chooseNOUN(p, tags)
                listofcombos.append(a)
            if pos.upper() == 'VERB':
                a = chooseVERB(p, tags)
                listofcombos.append(a)
            if pos.upper() == 'GRND':
                a = chooseGRND(p, tags)
                listofcombos.append(a)
    for combo in listofcombos:
        for i in combo:
            if i:
                DFcolind.append(str(i))
    DFcolind = list(dict.fromkeys(DFcolind))
    
    return listofcombos, DFcolind

In [ ]:
def counting(listofcombos):
    
    counter = {}
    newlist = []
    for combo in listofcombos:
        for tagnumber in range(len(combo)-1):
            if combo[tagnumber] and combo[tagnumber+1]:
                pair = (combo[tagnumber],combo[tagnumber+1])
                newlist.append(pair)
    for pair in newlist:
        if pair in counter:
            counter[pair] += 1
        else:
            counter[pair] = 1
            
    return counter

In [ ]:
def heatmap(counter, DFcolind):
    
    DFcolindNN = []
    for i in DFcolind:
        if i:
            DFcolindNN.append(i)
    df = pd.DataFrame(columns = DFcolindNN, index = DFcolindNN)
    for key,value in counter.items():
        df.at[key[0],key[1]] = value
        df.at[key[1],key[0]] = value
    df = df.fillna(0)
    sns.heatmap(df)

In [ ]:
def main():
    
    h = inputlink()
    name = subst(h)
    text = import_posts(name)
    part, feats = choosePOS()
    morph = pm.MorphAnalyzer()
    allcombos, indexcol = listoftags(part, text, feats, morph)
    numbers = counting(allcombos)
    heatm = heatmap(numbers,indexcol)

In [ ]:
if __name__ == '__main__':
    
    main()